In [1]:
import os
import platform
from IPython.display import clear_output
print(platform.platform())

def resolve_dir(Dir):
    if not os.path.exists(Dir):
        os.mkdir(Dir)

def reset_path(Dir):
    if not os.path.exists(Dir):
        os.mkdir(Dir)
    else:
        os.system('rm -f {}/*'.format( Dir))

macOS-13.4.1-arm64-arm-64bit


In [2]:
import cv2
import os
import imageio
import imgaug.augmenters as iaa
import imgaug as ia

IMG_SIZE = 128
ColorChannels = 3

def video_to_frames(video):
    vidcap = cv2.VideoCapture(video)
    
    import math
    rate = math.floor(vidcap.get(3))
    count = 0
    
    ImageFrames = []
    while vidcap.isOpened():
        ID = vidcap.get(1)
        success, image = vidcap.read()
        
        if success:
            if (ID % 7 == 0):
                flip = iaa.Fliplr(1.0)
                zoom = iaa.Affine(scale=1.3)
                random_brightness = iaa.Multiply((1, 1.3))
                rotate = iaa.Affine(rotate=(-25, 25))
                
                image_aug = flip(image = image)
                image_aug = random_brightness(image = image_aug)
                image_aug = zoom(image = image_aug)
                image_aug = rotate(image = image_aug)
                
                rgb_img = cv2.cvtColor(image_aug, cv2.COLOR_BGR2RGB)
                resized = cv2.resize(rgb_img, (IMG_SIZE, IMG_SIZE))
                ImageFrames.append(resized)
            count+=1
        else:
            break
    
    vidcap.release()
    
    return ImageFrames

In [4]:
print('we have \n{} Violence videos \n{} NonViolence videos'.format(
              len(os.listdir('/Users/faizahkureshi/Desktop/GAIP Project/violencedataset/NonViolence')), 
              len(os.listdir('/Users/faizahkureshi/Desktop/GAIP Project/violencedataset/NonViolence'))))

we have 
1000 Violence videos 
1000 NonViolence videos


In [5]:
import tensorflow as tf
tf.random.set_seed(73)
TPU_INIT = False

if TPU_INIT:
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
        tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    
    except ValueError:
        raise BaseException('ERROR: Not connected to a TPU runtime!')
else:
    !nvidia-smi
;    
print("Tensorflow version " + tf.__version__)

zsh:1: command not found: nvidia-smi
Tensorflow version 2.13.0-rc2


In [6]:
%%time
#tqdm is used here to visulaize the progress of dataset creation
from tqdm import tqdm

VideoDataDir ='/Users/faizahkureshi/Desktop/GAIP Project/violencedataset'
print('we have \n{} Violence videos \n{} NonViolence videos'.format(
              len(os.listdir('/Users/faizahkureshi/Desktop/GAIP Project/violencedataset/NonViolence')), 
              len(os.listdir('/Users/faizahkureshi/Desktop/GAIP Project/violencedataset/NonViolence'))))

X_original = []
y_original = []
CLASSES = ["NonViolence", "Violence"]

for category in os.listdir(VideoDataDir):
    category_path = os.path.join(VideoDataDir, category)
    if os.path.isdir(category_path):
        class_num = CLASSES.index(category)
        try:
            video_list = [video for video in os.listdir(category_path)[:350] if video != '.DS_Store']
            for i, video in enumerate(tqdm(video_list)):
                video_path = os.path.join(category_path, video)
                frames = video_to_frames(video_path)
                for j, frame in enumerate(frames):
                    X_original.append(frame)
                    y_original.append(class_num)
        except OSError as e:
            print(f"Error processing directory: {category_path}. Skipping...")
            continue



we have 
1000 Violence videos 
1000 NonViolence videos


100%|██████████| 350/350 [00:45<00:00,  7.61it/s]

CPU times: user 2min 44s, sys: 5.14 s, total: 2min 49s
Wall time: 1min 4s


In [7]:
import numpy as np
X_original = np.array(X_original).reshape(-1 , IMG_SIZE * IMG_SIZE * 3)
y_original = np.array(y_original)
len(X_original)

13487

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

stratified_sample = StratifiedShuffleSplit(n_splits=2, test_size=0.3, random_state=73)

for train_index, test_index in stratified_sample.split(X_original, y_original):
    X_train, X_test = X_original[train_index], X_original[test_index]
    y_train, y_test = y_original[train_index], y_original[test_index]

X_train_nn = X_train.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255
X_test_nn = X_test.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255